In [1]:
# If not already installed, uncomment the next line:
# !pip install pymongo pandas

import pandas as pd
from pymongo import MongoClient, errors
import datetime
import re
from pathlib import Path

In [ ]:
def parse_filename(filename):
    """
    Parse filename like XAUUSD_M1_202502071057_202505221032.csv
    Returns: symbol, timeframe, start_str, end_str
    """
    pattern = r"([A-Z]+)_([A-Z0-9]+)_(\d{12})_(\d{12})\.csv"
    match = re.match(pattern, filename)
    if not match:
        raise ValueError("Filename does not match expected pattern")
    symbol, timeframe, start_str, end_str = match.groups()
    return symbol, timeframe, start_str, end_str

def str_to_datetime(dt_str):
    """Convert string like 202502071057 to datetime"""
    return datetime.datetime.strptime(dt_str, "%Y%m%d%H%M")

In [ ]:
mongo_uri = "mongodb://localhost:27017/"
db_name = "mt5_historical_data"
client = MongoClient(mongo_uri)
db = client[db_name]

In [ ]:
csv_path = "XAUUSD_M1_202502071057_202505221032.csv"  # Change as needed
symbol, timeframe, start_str, end_str = parse_filename(Path(csv_path).name)
start_dt = str_to_datetime(start_str)
end_dt = str_to_datetime(end_str)

# Read CSV
df = pd.read_csv(csv_path)
# If symbol is not in the CSV, add it
if 'symbol' not in df.columns:
    df['symbol'] = symbol

In [ ]:
collection_name = f"candles_{symbol}_{timeframe}"
col = db[collection_name]
col.create_index([("symbol", 1), ("time", 1)], unique=True)

inserted = 0
for record in df.to_dict(orient='records'):
    try:
        col.insert_one(record)
        inserted += 1
    except errors.DuplicateKeyError:
        pass  # Skip duplicates

print(f"Inserted {inserted} new documents into {collection_name}")

In [ ]:
meta_col = db["imported_ranges"]
meta_doc = {
    "symbol": symbol,
    "timeframe": timeframe,
    "start_datetime": start_dt,
    "end_datetime": end_dt,
    "filename": Path(csv_path).name,
    "imported_at": datetime.datetime.utcnow()
}
# Upsert: if a document with same symbol, timeframe, and filename exists, update it; else insert new
meta_col.update_one(
    {"symbol": symbol, "timeframe": timeframe, "filename": Path(csv_path).name},
    {"$set": meta_doc},
    upsert=True
)
print("Metadata updated.")

In [ ]:
for doc in meta_col.find({"symbol": symbol, "timeframe": timeframe}):
    print(doc)